In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

### Generate some synthetic data

In [ ]:
X = np.random.random([100, 1])
w_star = 2.0
noise = 0.2 * np.random.randn(100, 1)
y = X * w_star + noise
f = plt.scatter(X, y)

### Start with tensors for inputs

In [ ]:
inputs = tf.placeholder(tf.float32, shape = [100, 1], name = 'inputs')
targets = tf.placeholder(tf.float32, shape = [100, 1], name = 'targets')

### Declare variables

In [ ]:
w = tf.Variable(tf.random_normal([1, 1]), name = 'w')

### Define the model and loss

In [ ]:
preds = tf.matmul(inputs, w, name = 'preds')

# WARNING : Always check shapes when followed by shape agnostic operations
assert preds.get_shape() == targets.get_shape()

loss = tf.nn.l2_loss(preds - targets, name='loss')

### Define optimization steps

In [ ]:
learning_rate = 0.001
opt = tf.train.GradientDescentOptimizer(learning_rate)
grads = opt.compute_gradients(loss)
train_op = opt.apply_gradients(grads, name = 'train_op')

### Summaries

In [ ]:
# Summaries can be added anywhere the tensor we want to summarize is visible
tf.scalar_summary('train_loss', loss)
tf.scalar_summary('w', w[0, 0])

# Get a single op for all summaries
summary_op = tf.merge_all_summaries()  

# directory for writing summaries
summary_dir = '/tmp/lin_reg_summ/'

### Start training

In [ ]:
MAX_TRAIN_STEPS = 100
SUMMARY_STEPS = 1

init_op = tf.initialize_all_variables()
with tf.Session() as sess:
    # object that handles summary writing
    # We also give sess.graph to store the current computation graph
    summary_writer = tf.train.SummaryWriter(summary_dir, sess.graph)
    
    sess.run([init_op])
    feed_dict = {inputs : X,
                 targets: y}
    for step in range(0, MAX_TRAIN_STEPS):
        _, summary_str = sess.run([train_op, summary_op], feed_dict = feed_dict)
        if step % SUMMARY_STEPS == 0:
            summary_writer.add_summary(summary_str, step)  # Write to disk

Run tensorboard with 

`$ tensorboard --logdir='/tmp/lin_reg_summ/'`